### 라이브러리 불러오기

In [1]:
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# 동시 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
df = pd.read_csv('./data/attribution_data.csv')

# 데이터 보기
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/attribution_data.csv'

### 데이터 살펴보기 

* Cookie: 랜덤으로 생성된 고객 ID를 통해 다음 방문을 동일한 고객에게 연결
* Timestamp: 방문일시
* Interaction: 발생한 모든 작용
* Conversion: 전환 여부
* Conversion Value: 전환 이벤트 값
* Channel: 고객을 우리 사이트에 불러들인 마케팅 채널


In [ ]:
# 데이터 살펴보기
df.info()

In [ ]:
# 유니크한 key만 남기기 위해 중복 값 제거
# interaction 중복값 제거
df.interaction.drop_duplicates()

# channel 중복값 제거
df.channel.drop_duplicates()

In [ ]:
# 유저 별 unique 채널 수 구하기
df_channels_per_cookie = df.groupby('cookie').channel.nunique().reset_index()
df_channels_per_cookie.head()

In [ ]:
# 채널수 별 유저수(쿠키) 확인하기
df_channels_per_cookie.groupby('channel').count()

### 채널 기여도 분석

- Last Interaction Attribution Model
    - 상호작용한 마지막 채널이 전환가치에 100% 기여했다고 간주함

In [ ]:
df_last_interaction = df.drop_duplicates('cookie', keep='last')[['cookie', 'channel', 'conversion']]
print(len(df_last_interaction))
df_last_interaction.head(10)

In [ ]:
# 전환이 된 경우 마지막으로 접한 채널 확인
df_last_interaction_conv = df_last_interaction.loc[df_last_interaction.conversion == 1,:]
print("전환수",len(df_last_interaction_conv))
df_last_interaction_conv.head()

In [ ]:
# channel 별로 cookie 수를 확인
last_interaction_attrib = (
    df_last_interaction_conv
    .groupby('channel')['cookie']
    .count()
    .reset_index()
    .rename(columns={"cookie": "attribution"})
)
last_interaction_attrib

##### First Interaction Attribution Model
- 고객이 상호작용한 첫 번째 채널이 전환 가치에 100% 기여했다고 간주함

In [ ]:
# 유저가 처음으로 접한 채널만 추출
df_first_interaction = df.drop_duplicates('cookie', keep='first')[['cookie', 'channel', 'conversion']]
print(len(df_first_interaction))
df_first_interaction.head(10)

In [ ]:
# 전환 데이터와 merge 하자
df_first_interaction_conv = (
    df_first_interaction[['cookie', 'channel']]
      .merge(df_last_interaction_conv[['cookie', 'conversion']], 
             on='cookie')
)
df_first_interaction_conv.head()
print('전환 데이터:',len(df_first_interaction_conv))

In [ ]:
first_interaction_attrib = (
    df_first_interaction_conv
    .groupby('channel')['cookie']
    .count()
    .reset_index()
    .rename(columns={"cookie": "attribution"})
)
first_interaction_attrib

#### Linear Interaction Attribution Model

- 전환 경로에서 발생한 모든 채널의 기여도가 균등하다고 가정해보자
- 예를 들어 채널 1개에 노출되었다면 기여도 1, 2개에 노출되었다면 각 채널에 0.5, 0.5 씩 균등하게 배분하여 기여도를 계산한다.

In [ ]:
# 각 유저 별로 채널 별 데이터를 각 한 쌍만 남김
df_uniq_chl = df.drop_duplicates(['cookie', 'channel'], keep='last')[['cookie', 'channel']]
print(len(df_uniq_chl))
df_uniq_chl.head()

In [ ]:
# 유저 아이디별 채널 수를 구하고
# 기여도 분석을 위해
df_uniq_chl_cnt = (
    df_uniq_chl
    .groupby('cookie')
    .count()
    .rename(columns = {"channel": "channel_cnt"})
    .reset_index()
)
df_uniq_chl_cnt.head()

In [ ]:
# 앞서 구한 두 테이블을 합친다
df_uniq_chl_total = df_uniq_chl.merge(df_uniq_chl_cnt, on = 'cookie')
df_uniq_chl_total.head()

In [ ]:
# 전환 데이터와 merge 
df_linear_interaction_conv = (
    df_uniq_chl_total
      .merge(df_last_interaction_conv[['cookie', 'conversion']], 
             on='cookie')
)
len(df_linear_interaction_conv)

In [ ]:
# 전환을 채널수로 나누자
# 나누기 전에 데이터 타입을 확인하고 나눈다
df_linear_interaction_conv.info()

# 둘다 int 형 이므로 바로 나눠보자
df_linear_interaction_conv['linear_attr'] = (df_linear_interaction_conv.conversion.divide(df_linear_interaction_conv.channel_cnt))

df_linear_interaction_conv.head()

In [ ]:
# 채널별 기여도의 합을 구해보자 

linear_interaction_attrib = (
    df_linear_interaction_conv
    .groupby('channel')['linear_attr']
    .sum()
    .reset_index()
    .rename(columns={"linear_attr": "attribution"})
)
linear_interaction_attrib

#### 3가지 기여도 분석 방법 비교

- 앞에서 만들어둔 3가지 모델을 비교

In [ ]:
# 데이터를 하나로 합치기 위해
# method 컬럼을 만들어 각각 분류를 한다
last_interaction_attrib["method"] = "last"
first_interaction_attrib["method"] = "first"
linear_interaction_attrib["method"] = "linear"

In [ ]:
# 모델 확인
last_interaction_attrib
first_interaction_attrib
linear_interaction_attrib

In [ ]:
# 데이터 합치기
total_data = (
    last_interaction_attrib
    .append(first_interaction_attrib, ignore_index=True)
    .append(linear_interaction_attrib, ignore_index=True)
)

total_data

In [ ]:
# 시각화 하기
sns.set_theme(style="whitegrid", palette="Paired") 
g = sns.barplot(x=total_data.channel, y=total_data.attribution, hue = total_data.method)
g.figure.set_size_inches(10, 7)